In [1]:
# To Do:
# 1- delete mask and adjust
#!/usr/bin/env python
from tkinter import *
from PIL import Image, ImageEnhance
from PIL import ImageTk
from tkinter import filedialog
#from Tkinter import filedialog
import cv2

from scipy.misc import toimage
import numpy as np
import glob
#import GUI as segInter
import os
#import tkFont
from itertools import product
#f = tkFont.Font

#Color for Pixel modify


## Grabcut setting
BLUE = [255,0,0]        # rectangle color
RED = [0,0,255]         # PR BG
GREEN = [0,255,0]       # PR FG
BLACK = [0,0,0]         # sure BG
WHITE = [255,255,255]   # sure FG
YELLOW = [100,100,100]

# setting up flags

drawing = False         # flag for drawing curves

rect_over = False       # flag to check if rect drawn


thickness = 5 # brush thickness


DRAW_BG = {'color' : BLACK, 'val' : 0}
DRAW_FG = {'color' : WHITE, 'val' : 1}
DRAW_PR_FG = {'color' : GREEN, 'val' : 3}
DRAW_PR_BG = {'color' : RED, 'val' : 2}

Melanoma_SEGMENT_COLOR ={ 'ID' : 1,  'val' : 255}
Nevo_SSEGMENT_COLOR={'ID' : 2,  'val' : 128}
Unknwon_SEGMENT_COLOR ={'ID' : 3 , 'val' : 80}
trace = 0

# the images will be resized to this
width=768
height=576


class SegmentationGUI(Frame):
    def __init__(self,master):

	
        self.rect_or_mask = 100      # flag for selecting rect or mask mode        
        self.rect = (0,0,1,1)
        self.rectangle = False       # flag for drawing rect
        self.rect_over = False
        self.drawn=None
        self.pixelDrawn = None
        self.drawing = False
        self.add_mask = False
        #the name of the window
        master.title("MelanomaSeg Tool")
        self.panelA = None
        self.PixelValue = {'color' : None, 'val' : None}       # drawing initialized to None
        #vector of all images in the same folder
        self.images = None
        #self.masks = None
        #index of the image opened
        self.index_image=None
        self.index_mask=None
        #image opened 
        self.image=None
        self.image2=None
        self.imageToDrow= None
        self.grapOutput=None
        #the mask
        self.mask=None
        self.mask2 =None
        self.saveMask =None
        #path to the image
        self.path=None
        #path to mask
        #self.path_mask
        # Masks 
        self.uploadmask = None
        self.maskAux=None
        self.maskAll=None
        self.vectorMask=[]
        #image original before segmentation
        self.image_original=None
        self.height_original=None
        self.width_original=None

        #trace the actual zoom of the image
        self.zoomCycle = 0

        self.segment_type_choice=None
        self.segment_pixel_choice=None
        self.segment_class_choice=None
        self.mask_type_choice=None
        self.segments=None
        self.uploadMask= False
        self.color= None
        self.segment_class_color={}
        
        self.main_window()


    def main_window(self):
        # create a button, then when pressed, will trigger a file chooser
        # dialog and allow the user to select an input image; then add the
        # button the GUI
        btnSelectImage = Button(root, text="Open", command=self.open_image)
        btnBinaryMask = Button(root, text="Binary Mask", command= self.show_only_mask)
        #btnBinaryfinalMask = Button(root, text="final Binary Mask", command= self.show_final_mask)

        btnAddMask = Button(root, text="Add Mask", command = self.add_new_mask)
        btnShowImage = Button(root, text="Image", command= self.show_image_without_mask)
        btnShowMask = Button(root, text="Mask", command=self.show_image_with_mask)
        btnReset = Button(root, text="Reset", command=self.reset_image)
        #Next image button
        btnNext = Button(root, text='>>', command=self.Next)
        #Previous image button
        btnPrevious = Button(root, text='<<', command=self.Previous)
        btnuploadMask = Button(root, text="Upload Mask")
        # Canvas for show image       
        self.panelA = Canvas(root, width=900, height=600, background="bisque")
        # setting the radio btn
        self.segment_type_choice=IntVar()
        self.segment_pixel_choice=IntVar()
        self.segment_class_choice=IntVar()
        self.segment_type_choice.set(9)
        self.segment_pixel_choice.set(5)
        self.segment_class_choice.set(0)

        RadioButton1=Radiobutton(root, text="Grabcut",underline = True,padx = 20, variable=self.segment_type_choice, value=9,command=self.radioBtn_refresh)
        RadioButtonBG=Radiobutton(root, text="BG", padx = 20, variable=self.segment_pixel_choice, value=0,command=self.radioBtn_refresh)
        RadioButtonFG=Radiobutton(root, text="FG", padx = 20, variable=self.segment_pixel_choice, value=1,command=self.radioBtn_refresh)
        RadioButtonPBG=Radiobutton(root, text="PrBG", padx = 20, variable=self.segment_pixel_choice, value=2,command=self.radioBtn_refresh)
        RadioButtonPFG=Radiobutton(root, text="BrFG", padx = 20, variable=self.segment_pixel_choice, value=3,command=self.radioBtn_refresh)
        #RadioButtonUnknown=Radiobutton(root, text="Unknown", padx = 20, variable=self.segment_pixel_choice, value=2,command=self.radioBtn_refresh)
        RadioButtonMelanoma=Radiobutton(root, text="Melanoma", padx = 20, variable=self.segment_class_choice, value=11,command=self.class_refresh)
        RadioButtonNevo=Radiobutton(root, text="Nevo", padx = 20, variable=self.segment_class_choice, value=13,command=self.class_refresh)
        RadioButtonUnknown=Radiobutton(root, text="Atipici", padx = 20, variable=self.segment_class_choice, value=12,command=self.class_refresh)
        #RadioButtonPBG=Radiobutton(root, text="PrFG", padx = 20, variable=self.segment_pixel_choice, value=3,command=self.radioBtn_refresh)
             
        btnSeg = Button(root, text='Segment', command= self.segment_approach)
        # Save Mask Button
        btnSave = Button(root, text='Save', command=self.save_All)
        # Adjust the Mask
        #btndeleteCurrentMask = Button(root, text='Delete', command= self.delete_current_mask)
        self.text = Text(root,width=100, height=1)
        #set the position of the widget in the window
        btnSelectImage.grid(row=0,column=0)
        #btnBinaryfinalMask.grid(row=0,column=5)
        #btndeleteCurrentMask.grid(row=0,column=5)
        btnBinaryMask.grid(row=0,column=3)
        btnAddMask.grid(row=6,column=1)
        btnSave.grid(row=1,column=0)
        btnShowImage.grid(row=0,column=1)
        btnShowMask.grid(row=0,column=2)
        btnReset.grid(row=2,column=0)
        self.panelA.grid(row=1,column=1,columnspan=4,rowspan=4)
        self.text.grid(row=5,column=1,columnspan=3,sticky='WE')
        btnNext.grid(row=3,column=5)
        btnPrevious.grid(row=3,column=0)
        RadioButton1.grid(row=1,column=6)
        RadioButtonBG.grid(row=2,column=6)
        RadioButtonMelanoma.grid(row=6,column=2)
        RadioButtonNevo.grid(row=6,column=3)
        RadioButtonUnknown.grid(row=6,column=4)
        RadioButtonFG.grid(row=3,column=6)
        RadioButtonPBG.grid(row=2, column=7)
        RadioButtonPFG.grid(row=3, column=7)
        #RadioButtonPBG.grid(row=7,column=4)        
        btnSeg.grid(row=4,column=6)  
        btnuploadMask.grid(row=0,column=4)     
        # set the drowing param
        self.panelA.bind('<ButtonPress-1>', self.onStart) 
        self.panelA.bind('<B1-Motion>',     self.onGrow)
        self.panelA.bind('<ButtonRelease-1>', self.release)
        self.panelA.bind('<ButtonPress-3>', self.setPixels)
        self.panelA.bind('<ButtonRelease-3>', self.rightRelease)
        self.panelA.bind('<B3-Motion>', self.pixelOnGrown)    

        self.kinds = [self.panelA.create_rectangle, self.panelA.create_rectangle] 

    def onStart(self, event):
        self.class_refresh
        if ((self.add_mask == True) and ((self.segment_class_choice.get() == 11) or (self.segment_class_choice.get()==12)or(self.segment_class_choice.get()==13))) : 
         self.shape = self.panelA.create_rectangle
         self.kinds = self.panelA.create_rectangle 
         self.start = event
         self.drawn = None
         self.rectangle = True
        # Create Intermediate Mask and reset it when a new Rect is drawn
         self.mask = np.zeros(self.image.shape[:2], dtype = "uint8")
         self.mask2 = np.zeros(self.image.shape[:2], dtype = "uint8")  

      
    def onGrow(self, event):                         
       if self.rectangle == True:
        self.panelA = event.widget
       # Avoid duoblication
        if self.drawn: self.panelA.delete(self.drawn)
        objectId = self.shape(self.start.x, self.start.y, event.x, event.y,outline ="yellow")
        if trace: print (objectId)
        self.drawn = objectId
        #print self.panelA.bbox(objectId)
        self.rect = (self.panelA.bbox(objectId)[0],self.panelA.bbox(objectId)[1],self.panelA.bbox(objectId)[2],self.panelA.bbox(objectId)[3])
        
        self.rect_or_mask = 0
        #self.rectangle = False
    

    def release(self, event):
        self.rectangle = False
        self.rect_over = True  # Rect is drawn
        self.add_mask=False
       # self.segment_class_choice.set(0)


    def setPixels(self,event):
        self.radioBtn_refresh
        if  ((self.rect_over == False) or (self.segment_pixel_choice.get()==5)):
             print("First draw rectangle OR Set the pixel class ID \n")

        else:
            self.drawing = True
            self.pixelDrawn = None
            self.start = event
            cv2.circle(self.imageToDrow,(self.start.x,self.start.y),thickness,self.PixelValue['color'],-1)
            cv2.circle(self.mask,(self.start.x,self.start.y),thickness,self.PixelValue['val'],-1)
        #    print self.PixelValue['color'], self.PixelValue['val']
            pixelId = self.panelA.create_oval (self.start.x-1, self.start.y-1,self.start.x+1, self.start.y+1,outline = self.color,fill= self.color, width=3) 
            #print  "start", self.start.x,self.start.y


    def pixelOnGrown (self,event):
        self.radioBtn_refresh
        if self.drawing == True:
            self.panelA = event.widget
            #self.pixshape = self.PanelA.create_oval
            cv2.circle(self.imageToDrow,(event.x,event.y),thickness,self.PixelValue['color'],-1)
            cv2.circle(self.mask,(event.x,event.y),thickness,self.PixelValue['val'],-1)
            #print self.PixelValue['color'], self.PixelValue['val']
            #print event.x, event.y
            pixelId = self.panelA.create_oval (event.x-1, event.y-1,event.x+1, event.y+1,outline = self.color,fill= self.color, width=3) 


    def rightRelease (self,event):
        self.radioBtn_refresh
        if self.drawing == True:
            self.drawing = False
            cv2.circle(self.imageToDrow,(self.start.x,self.start.y),thickness,self.PixelValue['color'],-1)
            cv2.circle(self.mask,(self.start.x,self.start.y),thickness,self.PixelValue['val'],-1)
           # print self.PixelValue['color'] , self.PixelValue['val']


    def open_image(self):

        path=self.path_to_image()
        self.path=path
        self.rect_or_mask = 100      # flag for selecting rect or mask mode        
        # ensure a file path was selected
        if len(path) > 0:
            #get the root of the path
            pathRoot=path[:path.rfind('/')] + '/*'+path[path.rfind('.'):]
            #open all the images in the path
            self.images = glob.glob(pathRoot)
            #find the index of the image selected for trace the previous and next images
            self.index_image=self.find_index_selected(self.images,path)
            image = cv2.imread(path)
            # saving the original size of opened image
            self.image_original=image
            self.height_original,self.width_original=image.shape[:2]
            if (self.height_original > height or self.width_original > width):         
             image = cv2.resize(image,(width,height), interpolation = cv2.INTER_CUBIC)
             #print  'RESIZED' , self.image.shape[:2]
            #print  'image hegih' , self.height_original , 'widt&&', self.width_original
            #convert to RGB from BGR
            self.image2=image
            self.imageToDrow=image.copy()
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            self.image=image
            #self.image2=image
            #init the mask for the image
            self.mask = np.zeros(self.image.shape[:2], dtype = "uint8")
            self.mask2=self.mask
            self.saveMask = self.mask

            self.maskAll=self.mask

            self.maskAux=self.mask
            auximage = image 
           # convert the images to PIL format...
            auximage=toimage(auximage)
            # ...and then to ImageTk format
            imageOUT = ImageTk.PhotoImage(auximage)
            self.panelA.create_image(0, 0, image = imageOUT, anchor = NW)
            self.panelA.image = imageOUT
            self.text.delete(1.0,END)
            self.text.insert(INSERT,self.path)
 


    def path_to_image(self):
        path = filedialog.askopenfilename()
        return path
    
    #find the index of the actual image selected
    def find_index_selected(self,images,path):
        i=0
        indexSelected =0
        for pathImage in self.images:
            if(path==pathImage):
                indexSelected=i
            i=i+1
        return indexSelected


    def reset_image(self):
        self.panelA.delete('all')
        self.rect_or_mask = 100      # flag for selecting rect or mask mode        
        if(self.path!=None):
            path=self.path
            if len(path) > 0:
                #get the root of the path
                pathRoot=path[:path.rfind('/')] + '/*'+path[path.rfind('.'):]
                #open all the images in the path
                self.images = glob.glob(pathRoot)
                #find the index of the image selected for trace the previous and next images
                self.index_image=self.find_index_selected(self.images,path)
                image = cv2.imread(path)
                self.height_original,self.width_original=image.shape[:2]
                self.image_original=image
                # to fit the image into 
                if (self.height_original > height or self.width_original > width):         
                 image = cv2.resize(image,(width,height), interpolation = cv2.INTER_CUBIC)
                #convert to RGB from BGR
                self.image2=image
                self.imageToDrow=image.copy()
                self.grapOutput = image
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                self.image=image
                #init the mask for the image
                self.mask = np.zeros(self.image.shape[:2], dtype = "uint8")
                self.mask2=self.mask
                self.saveMask = self.mask
                self.maskAll=self.mask
                
                self.maskAux=self.mask
                # convert the images to PIL format...
                image=toimage(image)
                # ...and then to ImageTk format
                imageOUT = ImageTk.PhotoImage(image)
                self.panelA.create_image(0, 0, image = imageOUT, anchor = NW)
                self.panelA.image = imageOUT
                self.text.delete(1.0,END)
                self.text.insert(INSERT,self.path)


    def Next(self):
        self.rect_or_mask = 100      # flag for selecting rect or mask mode        
        if(self.path!=None):
            if(self.index_image<len(self.images)-1):
                self.index_image=self.index_image+1
                self.path=self.images[self.index_image]
                image = cv2.imread(self.path)
                self.height_original,self.width_original=image.shape[:2]
                self.image_original=image
                if (self.height_original > height or self.width_original > width):         
                 image = cv2.resize(image,(width,height), interpolation = cv2.INTER_CUBIC)
                 #print  'RESIZED' , self.image.shape[:2]
                #image = self.increase_contrast_image(image)
                #convert to RGB from BGR
                self.image2=image
                self.grapOutput = image
                self.imageToDrow=image.copy()

                #init the mask for the image
                self.mask = np.zeros(self.image.shape[:2], dtype = "uint8")
                self.mask2=self.mask
                self.saveMask = self.mask
                self.maskAll=self.mask
                self.maskAux=self.mask
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)               
                self.image=image
                #print self.saveMask.shape[:2], self.maskAll.shape[:2]
                # convert the images to PIL format...
                image=toimage(image)
                # ...and then to ImageTk format
                imageOUT = ImageTk.PhotoImage(image)
                self.panelA.create_image(0, 0, image = imageOUT, anchor = NW)
                self.panelA.image = imageOUT
                self.text.delete(1.0,END)
                self.text.insert(INSERT,self.path)


    def Previous(self):
        self.rect_or_mask = 100      # flag for selecting rect or mask mode        
        if(self.path!=None):
            if(self.index_image>=0):
                self.index_image=self.index_image-1
                self.path=self.images[self.index_image]
                image = cv2.imread(self.path)
                self.height_original,self.width_original=image.shape[:2]
                self.image_original=image
                if (self.height_original > height or self.width_original > width):         
                 image = cv2.resize(image,(width,height), interpolation = cv2.INTER_CUBIC)
                 #print  'RESIZED' , self.image.shape[:2]
               # image = self.increase_contrast_image(image)
                #convert to RGB from BGR
                self.image2=image
                self.grapOutput = image
                self.imageToDrow=image.copy()

                #init the mask for the image
                self.mask = np.zeros(self.image.shape[:2], dtype = "uint8")
                self.mask2=self.mask
                self.saveMask = self.mask
                self.maskAll=self.mask
                self.maskAux=self.mask
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                self.image=image

                # convert the images to PIL format...
                image=toimage(image)
                # ...and then to ImageTk format
                imageOUT = ImageTk.PhotoImage(image)

                self.panelA.create_image(0, 0, image = imageOUT, anchor = NW)
                self.panelA.image = imageOUT
                self.text.delete(1.0,END)
                self.text.insert(INSERT,self.path)


    def radioBtn_refresh(self):
        if(self.segment_type_choice.get()==9):
            if(self.segment_pixel_choice.get()==0):
              self.PixelValue = DRAW_BG
              self.color = 'black'
            if(self.segment_pixel_choice.get()==1):       
              self.PixelValue = DRAW_FG
              self.color ='red'           
            if(self.segment_pixel_choice.get()==2):
              self.PixelValue = DRAW_PR_BG
              self.color = 'gray'
            if(self.segment_pixel_choice.get()==3):
              self.PixelValue = DRAW_PR_FG
              self.color= 'pink'


    def class_refresh( self): 
       if (self.segment_class_choice.get()==11):
          self.segment_class_color= Melanoma_SEGMENT_COLOR
       if (self.segment_class_choice.get()==12):
          self.segment_class_color= Unknwon_SEGMENT_COLOR
       if (self.segment_class_choice.get()==13):
          self.segment_class_color= Nevo_SSEGMENT_COLOR
        
    def segment_approach (self):
          
        if(self.segment_type_choice.get() == 9):
            if (self.rect_or_mask == 0):         # grabcut with rect
                bgdmodel = np.zeros((1,65),np.float64)
                fgdmodel = np.zeros((1,65),np.float64)
                cv2.grabCut(self.image2,self.mask,self.rect,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_RECT)
                self.rect_or_mask = 1
            elif self.rect_or_mask == 1:         # grabcut with mask
                bgdmodel = np.zeros((1,65),np.float64)
                fgdmodel = np.zeros((1,65),np.float64)
                #print "with mask"
                cv2.grabCut(self.image2,self.mask,self.rect,bgdmodel,fgdmodel,3,cv2.GC_INIT_WITH_MASK)
        
        self.class_refresh
       # print  'Before' , self.saveMask.shape[:2]

        self.saveMask = np.where((self.mask==1)+(self.mask==3),self.segment_class_color['ID'],0).astype('uint8')
       # print  'after' , self.saveMask.shape[:2]
        self.mask2 = np.where((self.mask==1)+(self.mask==3),self.segment_class_color['val'],0).astype('uint8')

        # For show image with mask
        self.mask2 = cv2.bitwise_not(self.mask2)
        # create the BG as image and copy mask over it
        fg = cv2.bitwise_or(self.image2, self.image2, mask=self.mask2)
        self.mask2 = cv2.bitwise_not(self.mask2)

        background = np.full(self.mask2.shape, 255).astype('uint8')
        background = np.stack([self.mask2]*3, axis=2)
        #bk =background
        self.grapOutput =  cv2.bitwise_or(fg,background)
        image_to_show = cv2.cvtColor(self.grapOutput, cv2.COLOR_BGR2RGB)
        image_to_show =toimage(image_to_show)
        imageGrapCutOUT = ImageTk.PhotoImage(image_to_show)
        self.panelA.create_image(0, 0, image = imageGrapCutOUT, anchor = NW)
        self.panelA.image = imageGrapCutOUT
        
    def show_image_without_mask(self):
            auximage = self.image 
           # convert the images to PIL format...
            auximage=toimage(auximage)
            # ...and then to ImageTk format
            imageOUT = ImageTk.PhotoImage(auximage)
            self.panelA.create_image(0, 0, image = imageOUT, anchor = NW)
            self.panelA.image = imageOUT

    def show_image_with_mask (self):
        image_to_show = cv2.cvtColor(self.grapOutput, cv2.COLOR_BGR2RGB)
        image_to_show =toimage(image_to_show)
        imageGrapCutOUT = ImageTk.PhotoImage(image_to_show)
        self.panelA.create_image(0, 0, image = imageGrapCutOUT, anchor = NW)
        self.panelA.image = imageGrapCutOUT


    def delete_current_mask(self):
          self.mask = np.zeros(self.image.shape[:2], dtype = "uint8")
          self.mask2= self.mask
          self.grapOutput= self.image
          self.grapOutput= cv2.cvtColor(self.grapOutput, cv2.COLOR_BGR2RGB)
          self.rect_or_mask == 0
          self.rect_over == False
          self.drawing == False



    def show_only_mask(self):
      #  print "show only Binary Mask"
        # Show final MaskMelanoma
       # image_to_show = cv2.cvtColor(self.maskAll, cv2.COLOR_BGR2RGB)

        image_to_show =toimage(self.mask2)
        imageGrapCutOUT = ImageTk.PhotoImage(image_to_show )
        self.panelA.create_image(0, 0, image = imageGrapCutOUT, anchor = NW)
        self.panelA.image = imageGrapCutOUT

#self.grapOutput = cv2.bitwise_and(self.image2,self.image2,mask=self.mask2)
        #Fname=(self.path[self.path.rfind('/')+1 : self.path.rfind('.')] + 'MaskMelanoma')
        #f =  filedialog.asksaveasfile(mode='wb',initialfile=Fname, defaultextension=".png", filetypes=(("PNG file", "*.png"),("All Files", "*.*")))
        #if f:
         #   abs_path = os.path.abspath(f.name)
          #  cv2.imwrite(abs_path,self.grapOutput)


    #def show_final_mask(self):
   #     print "show final Binary Mask"
        # Show final MaskMelanoma
       # image_to_show = cv2.cvtColor(self.maskAll, cv2.COLOR_BGR2RGB)
    #    self.maskAll=np.where(self.saveMask == self.segment_class_color['ID'], self.segment_class_color['val'], self.maskAll)
    #    image_to_show =toimage(self.maskAll)
    #    imageGrapCutOUT = ImageTk.PhotoImage(image_to_show )
    #    self.panelA.create_image(0, 0, image = imageGrapCutOUT, anchor = NW)
      #  self.panelA.image = imageGrapCutOUT     
   
       
           
    def add_new_mask (self):
         # here save the last update of the mask based on the radio btn of each class (need to be added)

       self.save_mask (self.maskAll)
       #print "new Mask is added"
       #self.maskAll=np.where(self.saveMask == self.segment_class_color['ID'], self.segment_class_color['val'], self.maskAll)
       self.class_refresh
       self.radioBtn_refresh
       self.add_mask = True
       self.panelA.delete(self.drawn)

#       self.maskAll = cv2.bitwise_or(self.maskAll,self.mask2)

    def save_mask(self, mask_image):

        
       # self.maskAux = cv2.resize(self.maskAux,(self.width_original,self.height_original), interpolation = cv2.INTER_LANCZOS4)
        #print  'MASK ALL',mask_image.shape[:2]
        #print  'saveMask',self.saveMask.shape[:2]
        self.maskAll = cv2.bitwise_or (self.maskAll,self.saveMask )
       # print mask_image
        #mask_image=self.maskAux
        for i in range(self.maskAll.shape[0]):
              for j in range(self.maskAll.shape[1]):
                  pixel = self.maskAll.item(i, j)
                  if pixel == 1: 
                     mask_image [i,j] =255
                  if pixel == 2: 
                     mask_image [i,j] =190
                  if pixel == 3:
                     mask_image[i,j] = 50

    def save_All (self):
       #print 'save the final mask'
       self.save_mask(self.maskAux)
       imagetoshow = self.maskAux
       image_to_show =toimage(imagetoshow)
       imageGrapCutOUT = ImageTk.PhotoImage(image_to_show )
       self.panelA.create_image(0, 0, image = imageGrapCutOUT, anchor = NW)
       self.panelA.image = imageGrapCutOUT  


       if(self.path!=None):
            Fname=(self.path[self.path.rfind('/')+1 : self.path.rfind('.')])
            #print Fname

            f =  filedialog.asksaveasfile(mode='wb',initialfile=Fname, defaultextension=".png", filetypes=(("PNG file", "*.png"),("All Files", "*.*")))

            if f:
                abs_path = os.path.abspath(f.name)
                cv2.imwrite(abs_path,self.maskAll)
                #maskOUT=toimage(mask2saveWater)
                #maskOUT.save(abs_path)


  #  def upload_mask(self):

# Flage to upload mask2 
     #   self.uploadMask = True
     #   path=self.path_to_image()
     #   self.path_mask=path
      #  self.rect_or_mask = 100      # flag for selecting rect or mask mode        
        # ensure a file path was selected
      #  if len(path) > 0:
            #get the root of the path
     #       pathRoot=path[:path.rfind('/')] + '/*'+path[path.rfind('.'):]
            #open all the images in the path
       #       self.masks = glob.glob(pathRoot)
        #    #find the index of the image selected for trace the previous and next images
        #    self.index_mask=self.find_index_selected(self.masks,path)
        #    mask = cv2.imread(path)

#####################################################################

            # saving the original size of opened mask
      #      self.image_original=mask
      #      self.height_original,self.width_original=image.shape[:2]
      #      if (self.height_original > height or self.width_original > width):         
      #       mask = cv2.resize(image,(width,height), interpolation = cv2.INTER_CUBIC)

      #      print  'image hegih' , self.height_original , 'widt&&', self.width_original
           
# Here you set based on the uploaded mask 
            # set the mask all to the uploaded mask
       #     self.maskAll=self.uploadmask
      #      self.mask = np.zeros(self.upload_mask.shape[:2], dtype = "uint8")
       #     self.mask2=self.mask
      #      self.saveMask = self.mask
       #     self.maskAux=self.mask



# NOW you need to search for the image that match the mask
            #convert to RGB from BGR
       #     self.image2=image
       #     self.imageToDrow=image.copy()
       #     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      #      self.image=image
            #self.image2=image
            #init the mask for the image
            
      #      auximage = image 
           # convert the images to PIL format...
      #      auximage=toimage(auximage)
            # ...and then to ImageTk format
      #      imageOUT = ImageTk.PhotoImage(auximage)
      #      self.panelA.create_image(0, 0, image = imageOUT, anchor = NW)
      #      self.panelA.image = imageOUT
      #      self.text.delete(1.0,END)
       #     self.text.insert(INSERT,self.path)

             
      
#The master of the GUI
root=Tk()
#call the class for the SegmentationGUI
app=SegmentationGUI(root)
#start the GUI loop
root.mainloop()

/Users/wajdiahmed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:304: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/wajdiahmed/anaconda3/lib/python3.6/tkinter/__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-1-d77cacaed443>", line 486, in segment_approach
    self.saveMask = np.where((self.mask==1)+(self.mask==3),self.segment_class_color['ID'],0).astype('uint8')
KeyError: 'ID'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/wajdiahmed/anaconda3/lib/python3.6/tkinter/__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-1-d77cacaed443>", line 516, in show_image_with_mask
    image_to_show = cv2.cvtColor(self.grapOutput, cv2.COLOR_BGR2RGB)
cv2.error: OpenCV(3.4.2) /Users/trav